# Extract problematic frames from a label video

Let's say you have trained a model, analyzed and labeled video. By watching the label video, you might notice that you model is making some mistakes.

You can fix this by extracing the problematic frames, manually label the body parts and adding these images to your dataset, and training your model further.

This notebook helps with extracting new images for which you model is not performing well.

In [1]:
!pip install albumentations==1.3.0
!git clone https://github.com/kevin-allen/unetTracker
!pip install -r unetTracker/requirements.txt
!pip install -e unetTracker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1
Cloning into 'unetTracker'...
remote: Enumerating objects: 941, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 941 (delta 194), reused 309 (delta 188), pack-reused 621
Receiving objects: 100% (941/941), 123.86 MiB | 35.26 MiB/s, done.
Resolving deltas: 100% (584/584), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.0/380.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 6.8

Obtaining file:///content/unetTracker
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for unetTracker (pyproject.toml) ... done
  Created wheel for unetTracker: filename=unetTracker-0.0.1-0.editable-py3-none-any.whl size=16048 sha256=8ebd82f1fc08b789519fd0a9c3ac29cb5fb8f2260b25fa4d64640ed9bf6901cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-02b5rm4p/wheels/62/9b/5a/0cb547490a9187d698861d98e1e803c5e64f31a9d899a8e84c
Successfully built unetTracker


In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

fn = "/content/drive/My Drive/dsfolder"
if os.path.exists(fn):
  print("We can access the dsfolder directory.")
else:
  raise IOError("Problem accessing the dsfolder directory.")

Mounted at /content/drive
We can access the dsfolder directory.


In [2]:
# this will run the code in the setup_project.py and create a variable called `project`
fn = "/content/drive/My Drive/dsfolder/setup_project.py"
if os.path.exists(fn):
  print("We can access the file.")
else:
  raise IOError("Problem accessing the file.")

%run "/content/drive/My Drive/dsfolder/setup_project.py"

We can access the file.
Project directory: /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker
Getting configuration from config file. Values from config file will be used.
Loading /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker', 'normalization_values': {'means': [0.40666553378105164, 0.45300230383872986, 0.5148566961288452], 'stds': [0.23905879259109497, 0.2506697177886963, 0.2679229974746704]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}


In [3]:

%matplotlib widget
from unetTracker.dataset import UNetDataset
from unetTracker.unetGUI import LabelFromImagesGUI, SelectImagesFromLabeledVideoGUI

In [4]:
dataset = UNetDataset(image_dir=project.image_dir, mask_dir=project.mask_dir, coordinate_dir=project.coordinate_dir,
                      image_extension=project.image_extension)
print("Number of images in the dataset:",len(dataset))

Number of images in the dataset: 140


In [5]:
video_fn = '/content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/videos/human_hand.mp4'
label_fn = (os.path.splitext(video_fn)[0]) + "_label.avi"
print(video_fn, label_fn)

/content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/videos/human_hand.mp4 /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/videos/human_hand_label.avi


In [6]:
for fn in [video_fn,label_fn]:
    if os.path.exists(fn):
        print("Video file {} exist".format(fn))
    else:
        raise IOError("Video file {} does not exist".format(fn))

Video file /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/videos/human_hand.mp4 exist
Video file /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/videos/human_hand_label.avi exist


In [7]:
from google.colab import output
output.enable_custom_widget_manager()

In [8]:
myGui = SelectImagesFromLabeledVideoGUI(label_fn,
                                project)
display(myGui)

SelectImagesFromLabeledVideoGUI(children=(HBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x0…

In [9]:
myList = myGui.frame_list
print(len(myList))
myList[:10]

38


[322, 336, 342, 349, 364, 378, 508, 558, 564, 581]

In [10]:
extracted_frame_dir = project.project_dir+"/extracted_frames/" # directory where the extracted images will be saved
frame_info_file = extracted_frame_dir+"frame_info.csv" # file where the source of the image is stored



dataset.extract_frames_from_video(video_fn = video_fn,
                                  frame_dir = extracted_frame_dir,
                                  image_size = project.image_size,
                                  selected_frames = myList,
                                  frame_info_file = frame_info_file)

video length: 5795, image size: 270h 480w
Extracting frames: [322, 336, 342, 349, 364, 378, 508, 558, 564, 578, 581, 608, 614, 783, 827, 841, 950, 966, 1087, 1091, 1143, 1189, 1212, 1240, 1332, 1345, 1439, 1519, 1538, 1576, 1600, 1619, 1627, 1730, 1735, 1755, 1776, 1827] to /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/extracted_frames/
Saving image info to /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/extracted_frames/frame_info.csv


In [11]:
LabelFromImagesGUI(image_dir=extracted_frame_dir,
                   frame_info_file=frame_info_file,
                   project=project,dataset=dataset)

frame.shape: (270, 480, 3)
68e83520-9119-11ee-bc83-0242ac1c000c.png


LabelFromImagesGUI(children=(HTML(value='Event info'), HBox(children=(Label(value='Objects:'), RadioButtons(la…

In [ ]:
len(dataset)

200

Now that you have added problematic frames to your dataset, you can use them to train your network further.

You will have to go through Notebooks 04_ to 06_ to retrain your network.